In [1]:
import pandas as pd
from collections import defaultdict
import warnings
import visualize
import neat
import multiprocessing
import os
import matplotlib.pyplot as plt

import numpy as np

In [2]:
necoc = 30
generations = int(3000/necoc)
generations

100

In [3]:
def softmax(x, axis=1):
    # 计算每行的最大值
    row_max = x.max(axis=axis)
 
    # 每行元素都需要减去对应的最大值，否则求exp(x)会溢出，导致inf情况
    row_max=row_max.reshape(-1, 1)
    x = x - row_max
 
    # 计算e的指数次幂
    x_exp = np.exp(x)
    x_sum = np.sum(x_exp, axis=axis, keepdims=True)
    s = x_exp / x_sum
    return s

In [4]:
df = pd.read_csv('letter-recognition.data',header=None)

train_df = df[:1600]
test_df = df[16000:16000+400]

X = train_df.ix[:,1:16].to_numpy() 
y = np.array([ord(l)-ord('A') for l in train_df.ix[:,0].to_numpy()])

In [5]:
def get_winner(class1):
    
    numbers = [*range(26)] # number 2 > number 1
    number_of_classification = len(numbers)
    #number_of_sampling = 100

    list_y = y.tolist()
    digits_indexes = []
    for digit in numbers:
        li = [i for i in range(len(list_y)) if list_y[i] == digit]
        digits_indexes.extend([li])

        
    class2 = []

    class1_indexes = []
    class2_indexes = []

    for i in numbers:
        if i in class1:
            class1_indexes.extend(digits_indexes[i])
        else:
            class2_indexes.extend(digits_indexes[i])
            class2.append(i)

    class1_label = [1] * len(class1_indexes)
    class2_label = [0] * len(class2_indexes)

    print("number of class1:{}".format(len(class1_label)))
    print("number of class2:{}".format(len(class2_label)))

    samplesize = 1600
    x_inputs =[tuple(c) for c in X[class1_indexes].tolist()] + [tuple(c) for c in X[class2_indexes].tolist()]
    x_outputs = [tuple([c]) for c in class1_label + class2_label]
    
    
    def eval_genomes(genomes, config):
        for genome_id, genome in genomes:
            net = neat.nn.FeedForwardNetwork.create(genome, config)

            outputs = []
            for xi in x_inputs:
                output = net.activate(xi)
                outputs.append(output)


            px_outputs = softmax(np.array(outputs).reshape(samplesize, 2), axis=1)
            # the index of maximum in each line
            pred_outputs = np.argmax(px_outputs, axis = 1)
            real_outputs = np.array(x_outputs).reshape(samplesize,)

            acc = np.sum(pred_outputs == real_outputs)/samplesize

            genome.fitness = acc

    def run(config_file):
        # Load configuration.
        config = neat.Config(
            neat.DefaultGenome,
            neat.DefaultReproduction,
            neat.DefaultSpeciesSet,
            neat.DefaultStagnation,
            config_file,
        )

        # Create the population, which is the top-level object for a NEAT run.
        p = neat.Population(config)

        # add a stdout reporter to show progress in the terminal
        reporter = neat.StdOutReporter(False)
        p.add_reporter(reporter)
        stats = neat.StatisticsReporter()
        p.add_reporter(stats)
        #checkpointer = neat.Checkpointer(100)
        #p.add_reporter(checkpointer)
        # Run for up to 300 generations.
        winner = p.run(eval_genomes, generations)

        return [stats, winner]
    
    local_dir = os.getcwd()
    config_path = os.path.join(local_dir, "config-feedforward-letter2")
    [stats, winner] = run(config_path)
    
    return winner

In [6]:
import numpy as np
import random

number_codes = []
for i in range(2**(26-1), 2**26-1):
    if (i % 1000000) == 0:
            print(i)
    for j in bin(i).split('b')[1]:
        number_codes.append(int(j))
        


number_codes = np.array(number_codes).reshape(2**(26 - 1) - 1,26).T     

slice = random.sample(range(2**(26 - 1) - 1), necoc)

number_codes = number_codes[:,slice].tolist()

classes = []
for i in range(necoc):
    code_list = np.array(number_codes).T[i]
    classes.append(np.where(code_list==1)[0].tolist())

matrix_errors = []

for i in range(25):
    for j in range(i+1,26):
        matrix_errors.append(necoc - np.sum(np.array(number_codes[i]) == np.array(number_codes[j])))
min_error = np.min(matrix_errors)

print(min_error)

34000000
35000000
36000000
37000000
38000000
39000000
40000000
41000000
42000000
43000000
44000000
45000000
46000000
47000000
48000000
49000000
50000000
51000000
52000000
53000000
54000000
55000000
56000000
57000000
58000000
59000000
60000000
61000000
62000000
63000000
64000000
65000000
66000000
67000000
8


In [7]:
winner_list = []

for i,class1 in enumerate(classes):
    print("running:{}/30".format(i))
    winner = get_winner(class1)
    winner_list.append(winner)

running:0/30
number of class1:661
number of class2:939

 ****** Running generation 0 ****** 

Population's average fitness: 0.50072 stdev: 0.08091
Best fitness: 0.61062 - size: (2, 3) - species 4 - id 193
Average adjusted fitness: 0.118
Mean genetic distance 2.790, standard deviation 0.510
Population of 200 members in 22 species
Total extinctions: 0
Generation time: 2.939 sec

 ****** Running generation 1 ****** 

Population's average fitness: 0.53563 stdev: 0.07902
Best fitness: 0.63562 - size: (2, 4) - species 4 - id 272
Average adjusted fitness: 0.145
Mean genetic distance 2.901, standard deviation 0.532
Population of 201 members in 23 species
Total extinctions: 0
Generation time: 3.361 sec (3.150 average)

 ****** Running generation 2 ****** 

Population's average fitness: 0.53997 stdev: 0.07918
Best fitness: 0.64312 - size: (3, 5) - species 5 - id 531
Average adjusted fitness: 0.170
Mean genetic distance 2.989, standard deviation 0.547
Population of 203 members in 29 species
Total


 ****** Running generation 24 ****** 

Population's average fitness: 0.55402 stdev: 0.08244
Best fitness: 0.65375 - size: (5, 9) - species 24 - id 1119
Average adjusted fitness: 0.190
Mean genetic distance 3.031, standard deviation 0.531
Population of 197 members in 22 species
Total extinctions: 0
Generation time: 4.869 sec (4.583 average)

 ****** Running generation 25 ****** 

Population's average fitness: 0.56070 stdev: 0.07997
Best fitness: 0.65375 - size: (5, 9) - species 24 - id 1119
Average adjusted fitness: 0.194
Mean genetic distance 3.054, standard deviation 0.535
Population of 208 members in 21 species
Total extinctions: 0
Generation time: 4.619 sec (4.555 average)

 ****** Running generation 26 ****** 

Population's average fitness: 0.55924 stdev: 0.08202
Best fitness: 0.65375 - size: (5, 9) - species 24 - id 1119
Average adjusted fitness: 0.211
Mean genetic distance 3.045, standard deviation 0.549
Population of 196 members in 19 species
Total extinctions: 0
Generation tim

Population's average fitness: 0.55852 stdev: 0.08616
Best fitness: 0.66375 - size: (9, 18) - species 47 - id 6849
Average adjusted fitness: 0.165
Mean genetic distance 3.217, standard deviation 0.535
Population of 196 members in 21 species
Total extinctions: 0
Generation time: 5.302 sec (5.317 average)

 ****** Running generation 49 ****** 

Population's average fitness: 0.56731 stdev: 0.08396
Best fitness: 0.67188 - size: (10, 20) - species 47 - id 7354
Average adjusted fitness: 0.185
Mean genetic distance 3.224, standard deviation 0.549
Population of 203 members in 20 species
Total extinctions: 0
Generation time: 5.603 sec (5.371 average)

 ****** Running generation 50 ****** 

Population's average fitness: 0.55426 stdev: 0.08982
Best fitness: 0.67188 - size: (10, 20) - species 47 - id 7354
Average adjusted fitness: 0.187
Mean genetic distance 3.243, standard deviation 0.565
Population of 200 members in 20 species
Total extinctions: 0
Generation time: 5.933 sec (5.446 average)

 ****

Population's average fitness: 0.57321 stdev: 0.08771
Best fitness: 0.67625 - size: (8, 16) - species 72 - id 8348
Average adjusted fitness: 0.203
Mean genetic distance 3.164, standard deviation 0.573
Population of 202 members in 18 species
Total extinctions: 0
Generation time: 7.722 sec (7.136 average)

 ****** Running generation 73 ****** 

Population's average fitness: 0.57625 stdev: 0.08429
Best fitness: 0.67625 - size: (8, 16) - species 72 - id 8348
Average adjusted fitness: 0.197
Mean genetic distance 3.164, standard deviation 0.580
Population of 198 members in 18 species
Total extinctions: 0
Generation time: 7.217 sec (7.228 average)

 ****** Running generation 74 ****** 

Population's average fitness: 0.58064 stdev: 0.08345
Best fitness: 0.67625 - size: (8, 16) - species 72 - id 8348
Average adjusted fitness: 0.224
Mean genetic distance 3.156, standard deviation 0.567
Population of 199 members in 18 species
Total extinctions: 0
Generation time: 7.154 sec (7.207 average)

 ******

Population's average fitness: 0.57610 stdev: 0.08560
Best fitness: 0.68063 - size: (17, 33) - species 85 - id 14190
Average adjusted fitness: 0.156
Mean genetic distance 3.128, standard deviation 0.575
Population of 201 members in 15 species
Total extinctions: 0
Generation time: 8.506 sec (9.391 average)

 ****** Running generation 97 ****** 

Population's average fitness: 0.58234 stdev: 0.07844
Best fitness: 0.68063 - size: (17, 33) - species 85 - id 14190
Average adjusted fitness: 0.172
Mean genetic distance 3.088, standard deviation 0.567
Population of 202 members in 15 species
Total extinctions: 0
Generation time: 9.270 sec (9.328 average)

 ****** Running generation 98 ****** 

Population's average fitness: 0.57041 stdev: 0.08859
Best fitness: 0.68063 - size: (17, 33) - species 85 - id 14190
Average adjusted fitness: 0.190
Mean genetic distance 3.082, standard deviation 0.531
Population of 203 members in 17 species
Total extinctions: 0
Generation time: 8.928 sec (9.160 average)

 

...

Population's average fitness: 0.51093 stdev: 0.02630
Best fitness: 0.57812 - size: (3, 5) - species 6 - id 297
Average adjusted fitness: 0.075
Mean genetic distance 2.704, standard deviation 0.531
Population of 200 members in 13 species
Total extinctions: 0
Generation time: 3.930 sec (3.351 average)

 ****** Running generation 2 ****** 

Population's average fitness: 0.51137 stdev: 0.03073
Best fitness: 0.58688 - size: (2, 4) - species 3 - id 445
Average adjusted fitness: 0.115
Mean genetic distance 2.808, standard deviation 0.543
Population of 199 members in 19 species
Total extinctions: 0
Generation time: 3.161 sec (3.288 average)

 ****** Running generation 3 ****** 

Population's average fitness: 0.51857 stdev: 0.03401
Best fitness: 0.60813 - size: (3, 6) - species 15 - id 706
Average adjusted fitness: 0.118
Mean genetic distance 2.831, standard deviation 0.540
Population of 202 members in 22 species
Total extinctions: 0
Generation time: 3.058 sec (3.230 average)

 ****** Running g

Population's average fitness: 0.53733 stdev: 0.04614
Best fitness: 0.62125 - size: (4, 6) - species 5 - id 3793
Average adjusted fitness: 0.138
Mean genetic distance 3.204, standard deviation 0.553
Population of 191 members in 25 species
Total extinctions: 0
Generation time: 4.498 sec (4.403 average)

 ****** Running generation 26 ****** 

Population's average fitness: 0.53439 stdev: 0.04528
Best fitness: 0.62125 - size: (4, 6) - species 5 - id 3793
Average adjusted fitness: 0.134
Mean genetic distance 3.254, standard deviation 0.557
Population of 205 members in 22 species
Total extinctions: 0
Generation time: 4.701 sec (4.437 average)

 ****** Running generation 27 ****** 

Population's average fitness: 0.52630 stdev: 0.04451
Best fitness: 0.62125 - size: (4, 6) - species 5 - id 3793
Average adjusted fitness: 0.118
Mean genetic distance 3.204, standard deviation 0.558
Population of 207 members in 20 species
Total extinctions: 0
Generation time: 4.897 sec (4.492 average)

 ****** Runni

Population's average fitness: 0.53819 stdev: 0.04887
Best fitness: 0.62125 - size: (4, 6) - species 5 - id 3793
Average adjusted fitness: 0.099
Mean genetic distance 3.061, standard deviation 0.605
Population of 195 members in 18 species
Total extinctions: 0
Generation time: 5.053 sec (5.168 average)

 ****** Running generation 50 ****** 

Population's average fitness: 0.53860 stdev: 0.05078
Best fitness: 0.62125 - size: (4, 6) - species 5 - id 3793
Average adjusted fitness: 0.118
Mean genetic distance 3.077, standard deviation 0.615
Population of 195 members in 18 species
Total extinctions: 0
Generation time: 5.405 sec (5.176 average)

 ****** Running generation 51 ****** 

Population's average fitness: 0.54145 stdev: 0.05079
Best fitness: 0.62125 - size: (4, 6) - species 5 - id 3793
Average adjusted fitness: 0.147
Mean genetic distance 3.087, standard deviation 0.602
Population of 201 members in 20 species
Total extinctions: 0
Generation time: 5.576 sec (5.217 average)

 ****** Runni

Total extinctions: 0
Generation time: 5.821 sec (6.671 average)

 ****** Running generation 74 ****** 

Population's average fitness: 0.53495 stdev: 0.04759
Best fitness: 0.62125 - size: (4, 6) - species 5 - id 3793
Average adjusted fitness: 0.092
Mean genetic distance 3.121, standard deviation 0.632
Population of 198 members in 17 species
Total extinctions: 0
Generation time: 6.209 sec (6.549 average)

 ****** Running generation 75 ****** 

Population's average fitness: 0.53869 stdev: 0.04410
Best fitness: 0.62125 - size: (4, 6) - species 5 - id 3793
Average adjusted fitness: 0.074
Mean genetic distance 3.091, standard deviation 0.644
Population of 202 members in 16 species
Total extinctions: 0
Generation time: 7.255 sec (6.659 average)

 ****** Running generation 76 ****** 

Population's average fitness: 0.53517 stdev: 0.04627
Best fitness: 0.62687 - size: (11, 22) - species 55 - id 12072
Average adjusted fitness: 0.131
Mean genetic distance 3.083, standard deviation 0.654
Population

Mean genetic distance 3.232, standard deviation 0.581
Population of 197 members in 18 species
Total extinctions: 0
Generation time: 10.061 sec (9.744 average)

 ****** Running generation 98 ****** 

Population's average fitness: 0.54082 stdev: 0.04460
Best fitness: 0.62875 - size: (12, 25) - species 66 - id 14746
Average adjusted fitness: 0.085
Mean genetic distance 3.218, standard deviation 0.585
Population of 197 members in 19 species
Total extinctions: 0
Generation time: 10.349 sec (9.829 average)

 ****** Running generation 99 ****** 

Population's average fitness: 0.53365 stdev: 0.04597
Best fitness: 0.62875 - size: (12, 25) - species 66 - id 14746
Average adjusted fitness: 0.101
Mean genetic distance 3.222, standard deviation 0.592
Population of 194 members in 19 species
Total extinctions: 0
Generation time: 11.033 sec (9.975 average)


In [9]:
testX = test_df.ix[:,1:16].to_numpy() 
testy = np.array([ord(l)-ord('A') for l in test_df.ix[:,0].to_numpy()])

local_dir = os.getcwd()
config_path = os.path.join(local_dir, "config-feedforward-letter2")

config = neat.Config(
    neat.DefaultGenome,
    neat.DefaultReproduction,
    neat.DefaultSpeciesSet,
    neat.DefaultStagnation,
    config_path,
)

def get_pred_real(i, j):
    winner_net = neat.nn.FeedForwardNetwork.create(winner_list[i], config)
    
    numbers = [*range(26)]
    class1 = classes[i]
    class2 = []
    class1_indexes = []
    class2_indexes = []
    
    test_X = testX[j]
    test_y = testy[j]

    list_y = [test_y]
    digits_indexes = []
    for digit in numbers:
        li = [i for i in range(len(list_y)) if list_y[i] == digit]
        digits_indexes.extend([li])

    for i in numbers:
        if i in class1:
            class1_indexes.extend(digits_indexes[i])
        else:
            class2_indexes.extend(digits_indexes[i])
            class2.append(i)

    class1_label = [1] * len(class1_indexes)
    class2_label = [0] * len(class2_indexes)


    testsamplesize = 1
    test_x_inputs =[tuple(test_X)]
    test_x_outputs = [tuple([c]) for c in class1_label + class2_label]

    outputs = []
    for xi in test_x_inputs:
        output = winner_net.activate(xi)
        outputs.append(output)

    px_outputs = softmax(np.array(outputs).reshape(testsamplesize, 2), axis=1)
    # the index of maximum in each line
    pred_outputs = np.argmax(px_outputs, axis = 1)
    real_outputs = np.array(test_x_outputs).reshape(testsamplesize,)

    
    return [pred_outputs, real_outputs]

from random import randint
pred_value = []
error_list = []

for j in range(400):
    pred = []
    for i in range(necoc):
        [pred_outputs, real_outputs] = get_pred_real(i, j)
        #print(pred_outputs, real_outputs)
        pred.append(pred_outputs)
    #print(np.array(pred).T)
    
    error = []
    for i in range(26):
        error.append(necoc - np.sum(number_codes[i] == np.array(pred).T))
    #print(error)
    
    pred_value.append(np.where(error==np.min(error)))
    error_list.append(np.min(error))
    #print(np.where(error==np.min(error)) )
list_P = []
for i in pred_value:
    if (len(i[0])) == 1:
        list_P.append(i[0][0])
    else:
        random_pick = randint(0, len(i[0])-1)
        list_P.append(i[0][random_pick])
np.sum(list_P == testy)/400

0.365

In [11]:
def get_pred_train(i, j):
    winner_net = neat.nn.FeedForwardNetwork.create(winner_list[i], config)
    
    numbers = [*range(26)]
    class1 = classes[i]
    class2 = []
    class1_indexes = []
    class2_indexes = []
    
    test_X = X[j]
    test_y = y[j]

    list_y = [test_y]
    digits_indexes = []
    for digit in numbers:
        li = [i for i in range(len(list_y)) if list_y[i] == digit]
        digits_indexes.extend([li])

    for i in numbers:
        if i in class1:
            class1_indexes.extend(digits_indexes[i])
        else:
            class2_indexes.extend(digits_indexes[i])
            class2.append(i)

    class1_label = [1] * len(class1_indexes)
    class2_label = [0] * len(class2_indexes)


    testsamplesize = 1
    test_x_inputs =[tuple(test_X)]
    test_x_outputs = [tuple([c]) for c in class1_label + class2_label]

    outputs = []
    for xi in test_x_inputs:
        output = winner_net.activate(xi)
        outputs.append(output)

    px_outputs = softmax(np.array(outputs).reshape(testsamplesize, 2), axis=1)
    # the index of maximum in each line
    pred_outputs = np.argmax(px_outputs, axis = 1)
    real_outputs = np.array(test_x_outputs).reshape(testsamplesize,)

    
    return [pred_outputs, real_outputs]

pred_value = []
error_list = []

for j in range(1600):
    pred = []
    for i in range(necoc):
        [pred_outputs, real_outputs] = get_pred_train(i, j)
        #print(pred_outputs, real_outputs)
        pred.append(pred_outputs)
    #print(np.array(pred).T)
    
    error = []
    for i in range(26):
        error.append(26 - np.sum(number_codes[i] == np.array(pred).T))
    #print(error)
    
    pred_value.append(np.where(error==np.min(error)))
    error_list.append(np.min(error))
    #print(np.where(error==np.min(error)) )

list_P = []
for i in pred_value:
    if (len(i[0])) == 1:
        list_P.append(i[0][0])
    else:
        random_pick = randint(0, len(i[0])-1)
        list_P.append(i[0][random_pick])
print("Trainset acc:{}".format(np.sum(list_P == y)/1600))
winner_fitness = []
for winner in winner_list:
    #print(winner.fitness)
    winner_fitness.append(winner.fitness)
print("Avg Base Learner:{}".format(np.mean(winner_fitness)))

Trainset acc:0.376875
Avg Base Learner:0.6952916666666665


In [12]:
# program to check if there is exist a path between two vertices 
# of a graph 

from collections import defaultdict 

#This class represents a directed graph using adjacency list representation 
class Graph: 

	def __init__(self,vertices): 
		self.V= vertices #No. of vertices 
		self.graph = defaultdict(list) # default dictionary to store graph 

	# function to add an edge to graph 
	def addEdge(self,u,v): 
		self.graph[u].append(v) 
	
	# Use BFS to check path between s and d 
	def isReachable(self, s, d): 
		# Mark all the vertices as not visited 
		visited =[False]*(self.V) 

		# Create a queue for BFS 
		queue=[] 

		# Mark the source node as visited and enqueue it 
		queue.append(s) 
		visited[s] = True

		while queue: 

			#Dequeue a vertex from queue 
			n = queue.pop(0) 
			
			# If this adjacent node is the destination node, 
			# then return true 
			if n == d: 
				return True

			# Else, continue to do BFS 
			for i in self.graph[n]: 
				if visited[i] == False: 
					queue.append(i) 
					visited[i] = True
		# If BFS is complete without visited d 
		return False
    
    
def findAllPath(graph,start,end,path=[]):
    path = path +[start]
    if start == end:
        return [path]
 
    paths = [] #存储所有路径    
    for node in graph[start]:
        if node not in path:
            newpaths = findAllPath(graph,node,end,path) 
            for newpath in newpaths:
                paths.append(newpath)
    return paths

numbers = [*range(2)]
number_of_classification = len(numbers)

list_nodes_number = []
list_connection_number = []

for winner in winner_list:
    used_nodes = list(winner.nodes.keys())
    # create a dict for mapping 
    l1 = numbers + config.genome_config.input_keys + used_nodes[number_of_classification:]
    l2 = range(len(l1))
    dict_nodes = dict(zip(l1, l2))

    ### 将节点加入图
    g = Graph(len(l1))# inputs + outputs + used  
    # add connections in the graph
    for cg in winner.connections.values():
        if cg.enabled:
            g.addEdge(dict_nodes[cg.key[0]],dict_nodes[cg.key[1]])

    v = numbers
    list_connections = []
    for vi in v:
        for u in range(number_of_classification, len(config.genome_config.input_keys) + number_of_classification):
            list_connections.append(g.isReachable(u, vi))

    nodes_inputs = range(number_of_classification, number_of_classification+ 64)
    nodes_outputs = range(number_of_classification)
    nodes_mid = range(number_of_classification + 64, len(dict_nodes))

    # 找到所有输入和输出连接的路
    all_path = []
    for u in nodes_inputs:
        for v in nodes_outputs:
            path = findAllPath(g.graph, u, v)
            if path:
                all_path = all_path + path

    # 得到最长路，以及每个节点在哪个层级
    max_length = max([len(x) for x in all_path])
    nodes_tuples_list = []
    for path in all_path:
        #print(path)
        for node in path:
            nodes_tuples_list.append([node, path.index(node)])

    # 确定节点的层级（消除重复）       
    nodes_tuples_fixed_list = []
    for index in range(max_length):
        for nodes in nodes_tuples_list:
            node, node_index = nodes[0], nodes[1]
            if node_index == index: 
                if nodes not in nodes_tuples_fixed_list:
                    nodes_tuples_fixed_list.append(nodes)


    # 所有有连接的节点
    all_used_nodes = []
    for node in nodes_tuples_fixed_list:
        if node[0] not in all_used_nodes:
            all_used_nodes.append(node[0])

    # 确定节点的层级（最终确定）
    nodes_tuples_fixed_list_final = []

    for node in all_used_nodes:
        if node in nodes_inputs:
            nodes_tuples_fixed_list_final.append([node, 0])
        elif node in nodes_outputs:
            nodes_tuples_fixed_list_final.append([node, max_length - 1])
        else:
            list_node = []
            for nodes in nodes_tuples_fixed_list:
                if node == nodes[0]:
                    list_node.append(nodes)
            layer = 0
            for ele in list_node:
                # 删除重复项，取最大
                if ele[1] > layer:
                    layer = ele[1]
            nodes_tuples_fixed_list_final.append([node,layer])


    # 每个层级的节点合数
    length_of_layers = []
    number_of_nodes = 0
    for index in range(max_length):
        for nodes in nodes_tuples_fixed_list_final:
            node, node_index = nodes[0], nodes[1]
            if node_index == index: 
                number_of_nodes += 1
        length_of_layers.append(number_of_nodes)

    length_of_each_layer = []
    length_of_each_layer.append(length_of_layers[0])
    for i in range(1, max_length):
        length_of_each_layer.append(length_of_layers[i] - length_of_layers[i-1])

    # 输出每个层级的节点个数
    #print("length of each layers:", length_of_each_layer)

    # 所有端到端的路
    all_path_side2side = []
    for path in all_path:
        if len(path) == 2:
            all_path_side2side.append(path)
        else:
            for i in range(len(path)-1):
                all_path_side2side.append([path[i],path[i+1]])

    # 定义节点到绘图的字典
    dict_nodes_graph = dict()
    count_number_layer = [0] * max_length
    for nodes in nodes_tuples_fixed_list_final:
        node, layer = nodes
        dict_nodes_graph[node] = [count_number_layer[layer], layer]
        count_number_layer[layer] += 1
    list_nodes_number.append(np.sum(length_of_each_layer))
    list_connection_number.append(len(all_path_side2side))
    #print("Number of nodes:{} Number of connections:{}".format(np.sum(length_of_each_layer),len(all_path_side2side)))
print("Total nodes:{} Total connections::{}".format(np.sum(list_nodes_number), np.sum(list_connection_number)))

Total nodes:470 Total connections::1815
